# Generating programs for all RosettaCode

In this script, we take the descriptions of all Rosetta Code programs and generate code for them. 

We use the Ollama framework for this task

The RosettaCode was pulled from this repo: https://github.com/acmeism/RosettaCodeData

In [1]:
# imports
import requests
import json
import os
import re
from tqdm import tqdm

In [2]:
# list all text files that contain text "TASK" in the filename from all subdirectories of ../RosettaCodeData
listTasks = []

for root, dirs, files in tqdm(os.walk('../RosettaCodeData/Task')):
    for file in files:
        if re.search(r'TASK', file):
            listTasks.append(os.path.join(root, file))

0it [00:00, ?it/s]

88168it [07:07, 206.20it/s]


In [7]:
def generate_code(task):
    # Define the API endpoint URL
    url = 'http://deeperthought.cse.chalmers.se:80/api/generate'
    
    # Prepare the data payload for the POST request
    data = {
        "model": "llama3.2",  # Specify the model to use
        "prompt": f"Write a python program to solve the task {task}. Generate only the code.",  # Define the prompt
        "stream": False  # Indicate whether to stream the response
    }
    
    # Set the headers for the POST request
    headers = {'Content-Type': 'application/json'}
    
    # Send the POST request to the API endpoint
    response = requests.post(url, data=json.dumps(data), headers=headers)
    
    # Parse the JSON response from the API
    json_data = json.loads(response.text)
    
    # Return the generated code from the response
    return json_data['response']

In [8]:
# now, take the test from each file and create the prompt for
# f'Write a python program to solve this task {task}'
# and send it to the model
import time

for task in tqdm(listTasks):
    with open(task, 'r') as f:
        # print(f'Processing: {task}')
        text = f.read()
        # extract the last two elements of the path of task
        task_filename_to_save = task.split('/')[-2:]
        task = task_filename_to_save[0] + '_' + task_filename_to_save[1].split('.')[0]
        # generate the code
        strCode = generate_code(text)
        # write this to the file in the folder ./rosetta_programs
        with open(f'./rosetta_programs/{task}_llama32.py', 'w') as f:
            f.write(strCode)
    # sleep for 3 seconds to avoid rate limiting
    time.sleep(3)

 85%|████████▌ | 1046/1228 [1:53:17<19:42,  6.50s/it] 


KeyboardInterrupt: 